In [ ]:
import mysql.connector

connection = mysql.connector.connect(
  host = "gateway01.ap-southeast-1.prod.aws.tidbcloud.com",
  port = 4000,
  user = "3odVFo6TfLUYgWX.root",
  password = "k3MbqRTW3UQ601wG"
  
)
cloudcursor = connection.cursor(buffered=True)
cloudcursor.execute("create database dataspark")

In [ ]:
!pip install pandas


[notice] A new release of pip is available: 24.2 -> 24.3.1
[notice] To update, run: python.exe -m pip install --upgrade pip


In [2]:
import pandas as pd
customer = pd.read_csv('f:/project2/DataSet-20240917T082626Z-001/DataSet/Customers.csv', encoding='latin1')


In [ ]:
import pandas as pd
customer = pd.read_csv('f:/project2/DataSet-20240917T082626Z-001/DataSet/Customers.csv', encoding='latin1')
# Ensure all rows and columns are displayed in the output
pd.set_option('display.max_rows', None)  # Show all rows
pd.set_option('display.max_columns', None)  # Show all columns
pd.set_option('display.width', None)  # Prevent line breaks
pd.set_option('display.colheader_justify', 'center')  # Center-align headers

# Display the filtered DataFrame
from IPython.display import display
display(customer)



In [24]:
print(customer.describe())
print(customer.info())
customer.shape

        CustomerKey
count  1.526600e+04
mean   1.060508e+06
std    6.127097e+05
min    3.010000e+02
25%    5.140335e+05
50%    1.079244e+06
75%    1.593980e+06
max    2.099937e+06
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 15266 entries, 0 to 15265
Data columns (total 10 columns):
 #   Column       Non-Null Count  Dtype 
---  ------       --------------  ----- 
 0   CustomerKey  15266 non-null  int64 
 1   Gender       15266 non-null  object
 2   Name         15266 non-null  object
 3   City         15266 non-null  object
 4   State Code   15256 non-null  object
 5   State        15266 non-null  object
 6   Zip Code     15266 non-null  object
 7   Country      15266 non-null  object
 8   Continent    15266 non-null  object
 9   Birthday     15266 non-null  object
dtypes: int64(1), object(9)
memory usage: 1.2+ MB
None


(15266, 10)

In [25]:
#Duplicate check in the whole data set

duplicate_rows = customer[customer.duplicated(keep=False)]
print(duplicate_rows)

Empty DataFrame
Columns: [CustomerKey, Gender, Name, City, State Code, State, Zip Code, Country, Continent, Birthday]
Index: []


In [3]:
customer.isnull().sum()


CustomerKey     0
Gender          0
Name            0
City            0
State Code     10
State           0
Zip Code        0
Country         0
Continent       0
Birthday        0
dtype: int64

In [52]:
# To show 10 state code null records

null_state_data = customer[customer['State Code'].isnull()]
print(null_state_data[['CustomerKey','State Code','State', 'Country']].head(10))

      CustomerKey State Code   State Country
5304       729681        NaN  Napoli   Italy
5316       732289        NaN  Napoli   Italy
5372       742042        NaN  Napoli   Italy
5377       742886        NaN  Napoli   Italy
5378       743343        NaN  Napoli   Italy
5485       759705        NaN  Napoli   Italy
5525       765589        NaN  Napoli   Italy
5531       766410        NaN  Napoli   Italy
5631       781667        NaN  Napoli   Italy
5695       789177        NaN  Napoli   Italy


In [27]:
napoli_data = customer[customer['State'] == 'Napoli']
print(napoli_data)

      CustomerKey  Gender                Name               City State Code  \
5304       729681  Female    Rossana Padovesi            Polvica        NaN   
5316       732289    Male        Indro Piccio          Varcaturo        NaN   
5372       742042    Male     Amaranto Loggia          Casaferro        NaN   
5377       742886  Female       Edmonda Capon           Terzigno        NaN   
5378       743343  Female        Ambra Sagese  Pomigliano D'Arco        NaN   
5485       759705    Male    Callisto Lo Duca            Casilli        NaN   
5525       765589    Male  Michelino Lucchesi       Pompei Scavi        NaN   
5531       766410    Male   Adelmio Beneventi             Licola        NaN   
5631       781667  Female          Ilda Manna             Napoli        NaN   
5695       789177    Male   Calogero Folliero      Mariglianella        NaN   

       State Zip Code Country Continent    Birthday  
5304  Napoli    80035   Italy    Europe   4/18/1981  
5316  Napoli    80014 

In [30]:

NA = customer[customer['State Code'] == 'NA']
print(NA)

      CustomerKey  Gender                Name               City State Code  \
5304       729681  Female    Rossana Padovesi            Polvica         NA   
5316       732289    Male        Indro Piccio          Varcaturo         NA   
5372       742042    Male     Amaranto Loggia          Casaferro         NA   
5377       742886  Female       Edmonda Capon           Terzigno         NA   
5378       743343  Female        Ambra Sagese  Pomigliano D'Arco         NA   
5485       759705    Male    Callisto Lo Duca            Casilli         NA   
5525       765589    Male  Michelino Lucchesi       Pompei Scavi         NA   
5531       766410    Male   Adelmio Beneventi             Licola         NA   
5631       781667  Female          Ilda Manna             Napoli         NA   
5695       789177    Male   Calogero Folliero      Mariglianella         NA   

       State Zip Code Country Continent    Birthday  
5304  Napoli    80035   Italy    Europe   4/18/1981  
5316  Napoli    80014 

In [29]:
# Clean the 'State' column
customer['State'] = customer['State'].str.strip().str.title()

# Define the mapping
state_to_code = {
    'Napoli': 'NA',
    # Add other mappings
}

# Replace null 'State Code' with mapped abbreviations
customer['State Code'] = customer.apply(
    lambda row: state_to_code.get(row['State'], row['State Code']) if pd.isnull(row['State Code']) else row['State Code'],
    axis=1
)

# Confirm the changes
print(customer.loc[customer['State'] == 'Napoli'])

# Save the cleaned data to a CSV file
output_path = 'cleaned_customer.csv'
customer.to_csv(output_path, index=False)
print(f"Cleaned data saved to {output_path}")


      CustomerKey  Gender                Name               City State Code  \
5304       729681  Female    Rossana Padovesi            Polvica         NA   
5316       732289    Male        Indro Piccio          Varcaturo         NA   
5372       742042    Male     Amaranto Loggia          Casaferro         NA   
5377       742886  Female       Edmonda Capon           Terzigno         NA   
5378       743343  Female        Ambra Sagese  Pomigliano D'Arco         NA   
5485       759705    Male    Callisto Lo Duca            Casilli         NA   
5525       765589    Male  Michelino Lucchesi       Pompei Scavi         NA   
5531       766410    Male   Adelmio Beneventi             Licola         NA   
5631       781667  Female          Ilda Manna             Napoli         NA   
5695       789177    Male   Calogero Folliero      Mariglianella         NA   

       State Zip Code Country Continent    Birthday  
5304  Napoli    80035   Italy    Europe   4/18/1981  
5316  Napoli    80014 

In [6]:
print(customer['State Code'].unique())


['SA' 'WA' 'VIC' 'QLD' 'NT' 'NSW' 'TAS' 'ACT' 'BC' 'QC' 'ON' 'AB' 'NS'
 'SK' 'NU' 'PE' 'MB' 'NL' 'YT' 'NB' 'BB' 'RP' 'BY' 'BW' 'NW' 'NI' 'ST'
 'MV' 'SN' 'TH' 'BE' 'HE' 'SL' 'SH' 'HB' 'HH' 'RA' 'IL' 'GY' 'AL' 'AQ'
 'CA' 'NP' 'GD' 'FC' 'PA' 'PC' 'LI' 'MP' 'PI' 'HN' 'AU' 'PL' 'LO' 'BN'
 'CE' 'LN' 'MQ' 'BO' 'BR' 'CO' 'MY' 'RC' 'VI' 'FE' 'RM' 'AG' 'IM' 'MI'
 'PR' 'BG' 'RG' 'PN' 'SV' 'LU' 'CN' 'TN' 'LE' 'PD' 'BI' 'CH' 'GE' 'TO'
 'VV' 'CZ' 'AN' 'PG' 'LT' 'BL' 'TV' 'PV' 'MN' 'VA' 'PT' 'SI' 'MS' 'CT'
 'BS' 'SS' 'RO' 'CR' 'FI' 'GR' 'IS' 'SO' 'VE' 'OR' 'ME' 'VR' 'CS' 'BZ'
 'NO' 'AV' 'TA' 'VC' 'GO' 'MO' 'FR' 'FG' 'TE' 'BA' 'UD' 'AP' 'TP' 'RE'
 'PO' 'NA' 'AT' 'SR' 'RI' 'TS' 'KR' 'MT' 'PZ' 'MC' 'VT' 'AR' 'AO' 'CB'
 'LC' 'SP' 'RN' 'FO' 'TR' 'UT' 'NH' 'DR' 'ZH' 'FL' 'OV' 'ZE' 'Falkirk'
 'Ceredigion' 'North East Lincolnshire' 'Aberdeenshire' 'York'
 'Pembrokeshire' 'Leicester' 'Highland' 'Tendring' 'Horsham' 'Newport'
 'Bristol' 'Newark and Sherwood' 'Argyllshire' 'Lincoln' 'Tamworth'
 'Fylde' 'Lewes' 

In [42]:
import pandas as pd
ft2=pd.read_csv("f:/project2/cleaned_Customer.csv")


In [8]:

print(customer.loc[customer['State'] == 'Napoli'])  # Confirm changes for Napoli


      CustomerKey  Gender                Name               City State Code  \
5304       729681  Female    Rossana Padovesi            Polvica         NA   
5316       732289    Male        Indro Piccio          Varcaturo         NA   
5372       742042    Male     Amaranto Loggia          Casaferro         NA   
5377       742886  Female       Edmonda Capon           Terzigno         NA   
5378       743343  Female        Ambra Sagese  Pomigliano D'Arco         NA   
5485       759705    Male    Callisto Lo Duca            Casilli         NA   
5525       765589    Male  Michelino Lucchesi       Pompei Scavi         NA   
5531       766410    Male   Adelmio Beneventi             Licola         NA   
5631       781667  Female          Ilda Manna             Napoli         NA   
5695       789177    Male   Calogero Folliero      Mariglianella         NA   

       State Zip Code Country Continent    Birthday  
5304  Napoli    80035   Italy    Europe   4/18/1981  
5316  Napoli    80014 

In [45]:
ft2.isnull().sum()

CustomerKey     0
Gender          0
Name            0
City            0
State Code     10
State           0
Zip Code        0
Country         0
Continent       0
Birthday        0
dtype: int64

In [57]:
# distinct state code ,state , coutry head of 50 records

customer[['State Code', 'State', 'Country']].drop_duplicates().head(50)

,State Code,State,Country
0,SA,South Australia,Australia
1,WA,Western Australia,Australia
2,VIC,Victoria,Australia
6,QLD,Queensland,Australia
8,NT,Northern Territory,Australia
12,NSW,New South Wales,Australia
26,TAS,Tasmania,Australia
69,ACT,Australian Capital Territory,Australia
1420,BC,British Columbia,Canada
1421,QC,Quebec,Canada


In [31]:
import pandas as pd
from datetime import datetime

# Convert 'Birthday' column to datetime format if it's not already
customer['Birthday'] = pd.to_datetime(customer['Birthday'], format='%m/%d/%Y', errors='coerce')

# Assuming 'Birthday' column is in 'MM-DD-YYYY' format
def calculate_age(birthdate):
  # Check if birthdate is already a Timestamp object
  if isinstance(birthdate, pd.Timestamp):
    birthdate_obj = birthdate
  else:
    try:
      birthdate_obj = datetime.strptime(birthdate, '%m-%d-%Y')
    except ValueError:
      return None
  today = datetime.today()
  age = today.year - birthdate_obj.year - ((today.month, today.day) < (birthdate_obj.month, birthdate_obj.day))
  return age

customer['Age'] = customer['Birthday'].apply(calculate_age)

customer.head(10)


,CustomerKey,Gender,Name,City,State Code,State,Zip Code,Country,Continent,Birthday,Age
0,301,Female,Lilly Harding,WANDEARAH EAST,SA,South Australia,5523,Australia,Australia,1939-07-03,85
1,325,Female,Madison Hull,MOUNT BUDD,WA,Western Australia,6522,Australia,Australia,1979-09-27,45
2,554,Female,Claire Ferres,WINJALLOK,VIC,Victoria,3380,Australia,Australia,1947-05-26,77
3,786,Male,Jai Poltpalingada,MIDDLE RIVER,SA,South Australia,5223,Australia,Australia,1957-09-17,67
4,1042,Male,Aidan Pankhurst,TAWONGA SOUTH,VIC,Victoria,3698,Australia,Australia,1965-11-19,59
5,1086,Male,Hayden Clegg,TEMPLERS,SA,South Australia,5371,Australia,Australia,1954-01-20,70
6,1133,Male,Nicholas Caffyn,JUBILEE POCKET,QLD,Queensland,4802,Australia,Australia,1969-11-22,55
7,1256,Male,Lincoln Jenks,KULLOGUM,QLD,Queensland,4660,Australia,Australia,1950-03-12,74
8,1314,Male,Isaac Israel,EDITH RIVER,NT,Northern Territory,852,Australia,Australia,1965-12-21,58
9,1568,Male,Luke Virtue,KOTTA,VIC,Victoria,3565,Australia,Australia,1975-07-25,49


In [10]:
import mysql.connector

connection = mysql.connector.connect(
  host = "gateway01.ap-southeast-1.prod.aws.tidbcloud.com",
  port = 4000,
  user = "3odVFo6TfLUYgWX.root",
  password = "k3MbqRTW3UQ601wG",
  database = "dataspark" 
  
)
cursor = connection.cursor(buffered=True)

In [6]:
import mysql.connector
# Define the SQL query

# Create a connection
try:
    connection = mysql.connector.connect(
  host = "gateway01.ap-southeast-1.prod.aws.tidbcloud.com",
  port = 4000,
  user = "3odVFo6TfLUYgWX.root",
  password = "k3MbqRTW3UQ601wG",
  database = "dataspark" 
  )

    # Create a cursor object
    cursor = connection.cursor()
    create_table_query = """
CREATE TABLE IF NOT EXISTS customer (
    CustomerKey INT,
    Gender VARCHAR(10),
    Name VARCHAR(255),
    City VARCHAR(255),
    StateCode VARCHAR(200),
    State VARCHAR(255),
    ZipCode VARCHAR(20),
    Country VARCHAR(255),
    Continent VARCHAR(255),
    Birthday DATE,
    Age INT
);
"""


    # Execute the query
    cursor.execute(create_table_query)

    # Commit the changes
    connection.commit()
    print("Table created successfully.")
    
except mysql.connector.Error as err:
    print(f"Error: {err}")
finally:
    if connection.is_connected():
        cursor.close()
        connection.close()
        print("MySQL connection is closed.")


Table created successfully.
MySQL connection is closed.


In [7]:
import mysql.connector
import pandas as pd
from datetime import datetime


# Load the CSV file
customer = pd.read_csv('f:/project2/DataSet-20240917T082626Z-001/DataSet/Customers.csv', encoding='latin1')

# Clean the 'State' column
customer['State'] = customer['State'].str.strip().str.title()

# Define the mapping
state_to_code = {
    'Napoli': 'NA',
    # Add other mappings
}

# Replace null 'State Code' with mapped abbreviations
customer['State Code'] = customer.apply(
    lambda row: state_to_code.get(row['State'], row['State Code']) if pd.isnull(row['State Code']) else row['State Code'],
    axis=1
)

# Step 2: Ensure there are no missing values in critical columns
customer = customer.dropna(subset=['CustomerKey', 'Name', 'State Code', 'Country'])  # Modify as needed



# Convert 'Birthday' column to datetime format if it's not already
customer['Birthday'] = pd.to_datetime(customer['Birthday'], format='%m/%d/%Y', errors='coerce')

# Assuming 'Birthday' column is in 'MM-DD-YYYY' format
def calculate_age(birthdate):
  # Check if birthdate is already a Timestamp object
  if isinstance(birthdate, pd.Timestamp):
    birthdate_obj = birthdate
  else:
    try:
      birthdate_obj = datetime.strptime(birthdate, '%m-%d-%Y')
    except ValueError:
      return None
  today = datetime.today()
  age = today.year - birthdate_obj.year - ((today.month, today.day) < (birthdate_obj.month, birthdate_obj.day))
  return age

customer['Age'] = customer['Birthday'].apply(calculate_age)


# Convert 'Birthday' column to datetime format if it's not already
#customer['Birthday'] = pd.to_datetime(customer['Birthday'], format='%m/%d/%Y', errors='coerce')

# Connect to the database
connection = mysql.connector.connect(
    host="gateway01.ap-southeast-1.prod.aws.tidbcloud.com",
    port=4000,
    user="3odVFo6TfLUYgWX.root",
    password="k3MbqRTW3UQ601wG",
    database="dataspark"
)
cursor = connection.cursor()

# Define the SQL Insert query
insert_query = """
INSERT INTO customer (CustomerKey, Gender, Name, City, StateCode, State, ZipCode, Country, Continent, Birthday,Age)
VALUES (%s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s)
"""

# Batch insert function
def batch_insert(cursor, connection, insert_query, data, batch_size=50):
    try:
        for i in range(0, len(data), batch_size):
            batch = data[i:i + batch_size]
            cursor.executemany(insert_query, batch)
            connection.commit()  # Commit after each batch
        print(f"{len(data)} records inserted successfully in batches of {batch_size}.")
    except mysql.connector.Error as err:
        print(f"Error inserting data in batches: {err}")
        connection.rollback()

# Prepare data for batch insertion
insert_data = [ (
            row['CustomerKey'],
            row['Gender'],
            row['Name'],
            row['City'],
            row['State Code'],  # Ensure the column names match
            row['State'],
            row['Zip Code'],    # Ensure the column names match
            row['Country'],
            row['Continent'],
            row['Birthday'].strftime('%Y-%m-%d') if pd.notna(row['Birthday']) else None,  # Convert to 'YYYY-MM-DD' format,
            row['Age']
        )
    for _, row in customer.iterrows()
]

# Insert data into the table
batch_insert(cursor, connection, insert_query, insert_data)

# Close the connection
cursor.close()
connection.close()




15266 records inserted successfully in batches of 50.


In [4]:
import pandas as pd
sale = pd.read_csv('f:/project2/DataSet-20240917T082626Z-001/DataSet/Sales.csv', encoding='latin1')
sale.head()

,Order Number,Line Item,Order Date,Delivery Date,CustomerKey,StoreKey,ProductKey,Quantity,Currency Code
0,366000,1,1/1/2016,NaN,265598,10,1304,1,CAD
1,366001,1,1/1/2016,1/13/2016,1269051,0,1048,2,USD
2,366001,2,1/1/2016,1/13/2016,1269051,0,2007,1,USD
3,366002,1,1/1/2016,1/12/2016,266019,0,1106,7,CAD
4,366002,2,1/1/2016,1/12/2016,266019,0,373,1,CAD


In [ ]:

print(sale.describe())
print(sale.info())
print(sale.isnull().sum())

       Order Number     Line Item   CustomerKey      StoreKey    ProductKey  \
count  6.288400e+04  62884.000000  6.288400e+04  62884.000000  62884.000000   
mean   1.430905e+06      2.164207  1.180797e+06     31.802144   1125.859344   
std    4.532963e+05      1.365170  5.859634e+05     22.978188    709.244010   
min    3.660000e+05      1.000000  3.010000e+02      0.000000      1.000000   
25%    1.121017e+06      1.000000  6.808580e+05      8.000000    437.000000   
50%    1.498016e+06      2.000000  1.261200e+06     37.000000   1358.000000   
75%    1.788010e+06      3.000000  1.686496e+06     53.000000   1650.000000   
max    2.243032e+06      7.000000  2.099937e+06     66.000000   2517.000000   

           Quantity  
count  62884.000000  
mean       3.144790  
std        2.256371  
min        1.000000  
25%        1.000000  
50%        2.000000  
75%        4.000000  
max       10.000000  
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 62884 entries, 0 to 62883
Data columns (

In [17]:

import mysql.connector

connection = mysql.connector.connect(
  host = "gateway01.ap-southeast-1.prod.aws.tidbcloud.com",
  port = 4000,
  user = "3odVFo6TfLUYgWX.root",
  password = "k3MbqRTW3UQ601wG",
  database = "dataspark" 
  
)
cursor = connection.cursor(buffered=True)


# Step 1: Create the table to match the columns of the 'sale' DataFrame
# Database connection

# SQL query to create a table
create_table_query = """
CREATE TABLE IF NOT EXISTS sale (
    OrderNumber INT,
    LineItem INT,
    OrderDate DATE,
    DeliveryDate DATE,
    CustomerKey INT,
    StoreKey INT,
    ProductKey INT,
    Quantity INT,
    CurrencyCode VARCHAR(10)
);
"""

try:
    # Create a cursor object
    cursor = connection.cursor()
    
    # Execute the query
    cursor.execute(create_table_query)
    print("Table created successfully.")
    
    # Commit changes (optional for DDL operations like CREATE TABLE)
    connection.commit()
    
except mysql.connector.Error as err:
    print(f"Error: {err}")
finally:
    # Close the cursor and connection
    if cursor:
        cursor.close()
    if connection:
        connection.close()




Table created successfully.


In [ ]:
import mysql.connector

# Step 1: Connect to the database
try:
    connection = mysql.connector.connect(
  host = "gateway01.ap-southeast-1.prod.aws.tidbcloud.com",
  port = 4000,
  user = "3odVFo6TfLUYgWX.root",
  password = "k3MbqRTW3UQ601wG",
  database = "dataspark" 
  )

    # Create a cursor object
    cursor = connection.cursor()

    print("Connected to the database successfully.")
except mysql.connector.Error as err:
    print(f"Error: {err}")
    exit()

# Step 2: Drop the table
drop_table_query = "DROP TABLE IF EXISTS customer;"  # Replace 'sale' with your table name

try:
    cursor.execute(drop_table_query)
    connection.commit()  # Not strictly required for DROP TABLE but good practice
    print("Table 'customer' dropped successfully.")
except mysql.connector.Error as err:
    print(f"Error dropping table: {err}")
finally:
    # Step 3: Close the connection
    if cursor:
        cursor.close()
    if connection:
        connection.close()


Connected to the database successfully.
Table 'cust' dropped successfully.


In [19]:
import mysql.connector
import pandas as pd

# Connect to the database
connection = mysql.connector.connect(
    host="gateway01.ap-southeast-1.prod.aws.tidbcloud.com",
    port=4000,
    user="3odVFo6TfLUYgWX.root",
    password="k3MbqRTW3UQ601wG",
    database="dataspark"
)
cursor = connection.cursor()

# Load the CSV file
sale = pd.read_csv(
    'f:/project2/DataSet-20240917T082626Z-001/DataSet/Sales.csv',
    encoding='latin1'
)

# Convert date columns to datetime
sale['Order Date'] = pd.to_datetime(sale['Order Date'], format='%m/%d/%Y', errors='coerce')
sale['Delivery Date'] = pd.to_datetime(sale['Delivery Date'], format='%m/%d/%Y', errors='coerce')

# Define the SQL Insert query
insert_query = """
INSERT INTO sale (
    OrderNumber, LineItem, OrderDate, DeliveryDate, CustomerKey, StoreKey, ProductKey, Quantity, CurrencyCode
) VALUES (%s, %s, %s, %s, %s, %s, %s, %s, %s)
"""

# Batch insert function
def batch_insert(cursor, connection, insert_query, data, batch_size=50):
    try:
        for i in range(0, len(data), batch_size):
            batch = data[i:i + batch_size]
            cursor.executemany(insert_query, batch)
            connection.commit()  # Commit after each batch
        print(f"{len(data)} records inserted successfully in batches of {batch_size}.")
    except mysql.connector.Error as err:
        print(f"Error inserting data in batches: {err}")
        connection.rollback()

# Prepare data for batch insertion
insert_data = [
    (
        row['Order Number'],
        row['Line Item'],
        row['Order Date'].strftime('%Y-%m-%d') if pd.notnull(row['Order Date']) else None,
        row['Delivery Date'].strftime('%Y-%m-%d') if pd.notnull(row['Delivery Date']) else None,
        row['CustomerKey'],
        row['StoreKey'],
        row['ProductKey'],
        row['Quantity'],
        row['Currency Code']
    )
    for _, row in sale.iterrows()
]

# Insert data into the table
batch_insert(cursor, connection, insert_query, insert_data)

# Close the connection
cursor.close()
connection.close()


62884 records inserted successfully in batches of 50.


In [2]:
import pandas as pd
product = pd.read_csv('f:/project2/DataSet-20240917T082626Z-001/DataSet/products.csv', encoding='latin1')
print(product.head())
product.shape

   ProductKey                         Product Name    Brand   Color  \
0           1  Contoso 512MB MP3 Player E51 Silver  Contoso  Silver   
1           2    Contoso 512MB MP3 Player E51 Blue  Contoso    Blue   
2           3     Contoso 1G MP3 Player E100 White  Contoso   White   
3           4    Contoso 2G MP3 Player E200 Silver  Contoso  Silver   
4           5       Contoso 2G MP3 Player E200 Red  Contoso     Red   

  Unit Cost USD Unit Price USD  SubcategoryKey Subcategory  CategoryKey  \
0        $6.62         $12.99              101     MP4&MP3            1   
1        $6.62         $12.99              101     MP4&MP3            1   
2        $7.40         $14.52              101     MP4&MP3            1   
3       $11.00         $21.57              101     MP4&MP3            1   
4       $11.00         $21.57              101     MP4&MP3            1   

  Category  
0    Audio  
1    Audio  
2    Audio  
3    Audio  
4    Audio  


(2517, 10)

In [4]:
print(product.describe())
print(product.info())

        ProductKey  SubcategoryKey  CategoryKey
count  2517.000000     2517.000000  2517.000000
mean   1259.000000      491.810091     4.878824
std     726.739637      229.887134     2.299170
min       1.000000      101.000000     1.000000
25%     630.000000      305.000000     3.000000
50%    1259.000000      406.000000     4.000000
75%    1888.000000      801.000000     8.000000
max    2517.000000      808.000000     8.000000
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2517 entries, 0 to 2516
Data columns (total 10 columns):
 #   Column          Non-Null Count  Dtype 
---  ------          --------------  ----- 
 0   ProductKey      2517 non-null   int64 
 1   Product Name    2517 non-null   object
 2   Brand           2517 non-null   object
 3   Color           2517 non-null   object
 4   Unit Cost USD   2517 non-null   object
 5   Unit Price USD  2517 non-null   object
 6   SubcategoryKey  2517 non-null   int64 
 7   Subcategory     2517 non-null   object
 8   CategoryKey     

In [5]:
#Duplicate check in the Cutomerkey field

duplicate_customer_product = product[product.duplicated(subset=['ProductKey'], keep=False)]
print(duplicate_customer_product)

Empty DataFrame
Columns: [ProductKey, Product Name, Brand, Color, Unit Cost USD, Unit Price USD, SubcategoryKey, Subcategory, CategoryKey, Category]
Index: []


In [3]:
#Null value any column
product.isnull().sum()

ProductKey        0
Product Name      0
Brand             0
Color             0
Unit Cost USD     0
Unit Price USD    0
SubcategoryKey    0
Subcategory       0
CategoryKey       0
Category          0
dtype: int64

In [4]:
# Clean and convert 'Unit Cost USD' and 'Unit Price USD' columns to numeric
for col in ['Unit Cost USD', 'Unit Price USD']:
    product[col] = (
        product[col].astype(str)  # Ensure string type
        .str.replace('$', '', regex=False)  # Remove '$' symbol
        .str.strip()  # Remove extra spaces
        .apply(pd.to_numeric, errors='coerce')  # Convert to numeric, coercing errors to NaN
        .fillna(0)  # Replace NaN with 0
    )

# Display the first few rows to verify changes
print(product.head())


   ProductKey                         Product Name    Brand   Color  \
0           1  Contoso 512MB MP3 Player E51 Silver  Contoso  Silver   
1           2    Contoso 512MB MP3 Player E51 Blue  Contoso    Blue   
2           3     Contoso 1G MP3 Player E100 White  Contoso   White   
3           4    Contoso 2G MP3 Player E200 Silver  Contoso  Silver   
4           5       Contoso 2G MP3 Player E200 Red  Contoso     Red   

   Unit Cost USD  Unit Price USD  SubcategoryKey Subcategory  CategoryKey  \
0           6.62           12.99             101     MP4&MP3            1   
1           6.62           12.99             101     MP4&MP3            1   
2           7.40           14.52             101     MP4&MP3            1   
3          11.00           21.57             101     MP4&MP3            1   
4          11.00           21.57             101     MP4&MP3            1   

  Category  
0    Audio  
1    Audio  
2    Audio  
3    Audio  
4    Audio  


In [8]:
import mysql.connector
import pandas as pd

# Load the CSV file
product = pd.read_csv('f:/project2/DataSet-20240917T082626Z-001/DataSet/products.csv', encoding='latin1')

# Clean and convert 'Unit Cost USD' and 'Unit Price USD' columns to numeric
for col in ['Unit Cost USD', 'Unit Price USD']:
    product[col] = (
        product[col].astype(str)  # Ensure string type
        .str.replace('$', '', regex=False)  # Remove '$' symbol
        .str.strip()  # Remove extra spaces
        .apply(pd.to_numeric, errors='coerce')  # Convert to numeric, coercing errors to NaN
        .fillna(0)  # Replace NaN with 0
    )


# Drop rows with missing critical values
product = product.dropna(subset=['ProductKey', 'Product Name', 'Unit Cost USD', 'Unit Price USD'])

# Connect to the database
connection = mysql.connector.connect(
    host="gateway01.ap-southeast-1.prod.aws.tidbcloud.com",
    port=4000,
    user="3odVFo6TfLUYgWX.root",
    password="k3MbqRTW3UQ601wG",
    database="dataspark"
)
cursor = connection.cursor()

# Step 1: Create the table if not exists
create_table_query = """
CREATE TABLE IF NOT EXISTS product (
    ProductKey INT,
    ProductName VARCHAR(255),
    Brand VARCHAR(255),
    Color VARCHAR(50),
    UnitCostUSD FLOAT,
    UnitPriceUSD FLOAT,
    SubcategoryKey INT,
    Subcategory VARCHAR(255),
    CategoryKey INT,
    Category VARCHAR(255)
);
"""
cursor.execute(create_table_query)

# Step 2: Insert data into the MySQL table
insert_query = """
INSERT INTO product (ProductKey, ProductName, Brand, Color, UnitCostUSD, UnitPriceUSD, SubcategoryKey, Subcategory, CategoryKey, Category)
VALUES (%s, %s, %s, %s, %s, %s, %s, %s, %s, %s)
"""
try:
    # Prepare a list of tuples for insertion
    values = [
        (
            row['ProductKey'],
            row['Product Name'],
            row['Brand'],
            row['Color'],
            row['Unit Cost USD'],
            row['Unit Price USD'],
            row['SubcategoryKey'],
            row['Subcategory'],
            row['CategoryKey'],
            row['Category']
        )
        for _, row in product.iterrows()
    ]

    # Use executemany for bulk insertion
    cursor.executemany(insert_query, values)

    # Commit the transaction
    connection.commit()
    print(cursor.rowcount, "record(s) inserted.")
except mysql.connector.Error as err:
    print(f"Error: {err}")
    connection.rollback()  # Rollback in case of error

# Close the cursor and connection
cursor.close()
connection.close()


2517 record(s) inserted.


In [1]:
import pandas as pd
store = pd.read_csv('f:/project2/DataSet-20240917T082626Z-001/DataSet/stores.csv', encoding='latin1')
print(store.head())
store.shape

   StoreKey    Country                         State  Square Meters  Open Date
0         1  Australia  Australian Capital Territory          595.0   1/1/2008
1         2  Australia            Northern Territory          665.0  1/12/2008
2         3  Australia               South Australia         2000.0   1/7/2012
3         4  Australia                      Tasmania         2000.0   1/1/2010
4         5  Australia                      Victoria         2000.0  12/9/2015


(67, 5)

In [9]:
import mysql.connector
import pandas as pd


store = pd.read_csv('f:/project2/DataSet-20240917T082626Z-001/DataSet/stores.csv', encoding='latin1')

# Ensure the 'Open Date' column is in a valid datetime format
store['Open Date'] = pd.to_datetime(store['Open Date'], format='%m/%d/%Y')

# Ensure there are no missing values in the DataFrame
store = store.dropna()  # Remove rows with NaN values

# Connect to the database
connection = mysql.connector.connect(
    host="gateway01.ap-southeast-1.prod.aws.tidbcloud.com",
    port=4000,
    user="3odVFo6TfLUYgWX.root",
    password="k3MbqRTW3UQ601wG",
    database="dataspark"
)
cursor = connection.cursor()

# Create the table for storing the data
cursor.execute("""
CREATE TABLE IF NOT EXISTS store (
    StoreKey INT PRIMARY KEY,
    Country VARCHAR(255),
    State VARCHAR(255),
    SquareMeters FLOAT,
    OpenDate DATE
);
""")


# Insert data from the DataFrame into the table
for index, row in store.iterrows():
    sql = "INSERT INTO store (StoreKey, Country, State, SquareMeters, OpenDate) VALUES (%s, %s, %s, %s, %s)"
    val = (row['StoreKey'], row['Country'], row['State'], row['Square Meters'], row['Open Date'].strftime('%Y-%m-%d'))
    cursor.execute(sql, val)

# Commit the transaction
connection.commit()

print(cursor.rowcount, "record(s) inserted.")



1 record(s) inserted.


In [10]:
import pandas as pd
er = pd.read_csv('f:/project2/DataSet-20240917T082626Z-001/DataSet/Exchange_Rates.csv', encoding='latin1')
print(er.head())
print(er.isnull().sum())
er.shape

       Date Currency  Exchange
0  1/1/2015      USD    1.0000
1  1/1/2015      CAD    1.1583
2  1/1/2015      AUD    1.2214
3  1/1/2015      EUR    0.8237
4  1/1/2015      GBP    0.6415
Date        0
Currency    0
Exchange    0
dtype: int64


(11215, 3)

In [ ]:
import mysql.connector
import pandas as pd

er = pd.read_csv('f:/project2/DataSet-20240917T082626Z-001/DataSet/Exchange_Rates.csv', encoding='latin1')


# Assuming the 'Date' column is in a valid datetime format
er['Date'] = pd.to_datetime(er['Date'], format='%m/%d/%Y')  # Ensure it's a datetime object

# Ensure there are no missing values in the DataFrame
er = er.dropna()  # Remove rows with NaN values

# Connect to the database
connection = mysql.connector.connect(
    host="gateway01.ap-southeast-1.prod.aws.tidbcloud.com",
    port=4000,
    user="3odVFo6TfLUYgWX.root",
    password="k3MbqRTW3UQ601wG",
    database="dataspark"
)
cursor = connection.cursor()

# Create the table for storing the data
cursor.execute("""
CREATE TABLE IF NOT EXISTS rate (
    rdate date,  
    Currency VARCHAR(255),
    Rate FLOAT
);
""")



# Insert data from the DataFrame into the table
for index, row in er.iterrows():
    sql = "INSERT INTO rate (rdate, Currency, Rate) VALUES (%s, %s, %s)"
    val = (row['Date'].strftime('%Y-%m-%d'), row['Currency'], row['Exchange'])  # Insert in 'YYYY-MM-DD' format
    cursor.execute(sql, val)

# Commit the transaction
connection.commit()

print(cursor.rowcount, "record(s) inserted.")







1 record(s) inserted.
